<div style="color:white;
            display:fill;
            border-radius:20px;
            background-color:green;
            font-size:200%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 10px;
               margin:0px auto 0px auto;
              color:white;
              text-align:center;'>
       &nbsp;Depth Prediction with MiDaS
    </h1>
    </div>

## This is the Reference code we have created and not the actual usable code. <br> 
### (Works on my system) (Needs different config for different computer)
#### One can easily edit the code for their paths and make it run for their system

In [ ]:
pip install timm

In [ ]:
model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

In [ ]:
import cv2
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
from torchvision import transforms

# Path to the folder containing YOLO annotated files
anno_folder = '/path/to/annotated/files'

# Path to the Midas depth image
midas_img_path = '/path/to/midas/image'

# Create an empty dictionary to store the results
results = {}

# Define the density of the material in cm^3/g (hand crafted)
density = {'Aluminium foil': 2.7,'Battery': 1.5,'Aluminium blister pack':2.71,'Carded blister pack': 2.4,'Other plastic bottle': 1.6,'Clear plastic bottle': 1,'Glass bottle': 3.2,'Plastic bottle cap': 0.92,'Metal bottle cap': 1.3,'Broken glass': 3.2,'Food Can': 3.6,'Aerosol': 2,'Drink can': 1,'Toilet tube': 2.2,'Other carton': 1.6,'Egg carton': 1.4,'Drink carton': 1.7,'Corrugated carton': 0.5,'Meal carton': 1.4,'Pizza box': 1.8,'Paper cup': 0.8,'Disposable plastic cup': 0.8,'Foam cup':0.6,'Glass cup': 2.2,'Other plastic cup': 0.8,'Food waste': 1.4,'Glass jar':2.4,'Plastic lid': 0.8,'Metal lid': 1.2,'Other plastic': 0.9,'Magazine paper': 0.3,'Tissues': 0.3,'Wrapping paper': 0.5,'Normal paper': 0.7,'Paper bag':0.7,'Plastified paper bag': 0.7,'Plastic film' : 1.2,'Six pack rings': 1.3,'Garbage bag': 4,'Other plastic wrapper' : 1.2,'Single-use carrier bag': 1.2,'Polypropylene bag': 1.3,'Crisp packet': 2.2,'Spread tub' : 3.4,'Tupperware' : 4.2,'Disposable food container':2.7,'Foam food container': 1.7,'Other plastic container': 1.7,'Plastic glooves': 2.2,'Plastic utensils': 1.9,'Pop tab': 0.4,'Rope & strings': 0.2,'Scrap metal' : 4.3,'Shoe' : 3.8,'Squeezable tube': 1.7,'Plastic straw' : 0.8,'Paper straw': 0.6,'Styrofoam piece' : 2.2,'Unlabeled litter': 2,'Cigarette': 4.2}

# Define a constant factor to scale the weight calculation (set according to comparitions of the results with the ground truth)
scale_factor = 0.001

# Initialize a variable to store the total weight of the image
total_weight = 0

# Loop through the annotated files in the folder
for filename in os.listdir(anno_folder):
    if filename.endswith('.jpg'):
        # Read input image
        img = cv2.imread(os.path.join(anno_folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        input_batch = transform(img).to(device)

        with torch.no_grad():
            prediction = midas(input_batch)

            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

        output = prediction.cpu().numpy()

        # Save output image
        save_filename = os.path.join(midas_img_path, os.path.splitext(filename)[0] + '_midas')
        plt.imsave(save_filename + '.png', output, cmap='gray')

    if filename.endswith('.txt'):
        # Load the YOLO annotation file
        anno_file = os.path.join(anno_folder, filename)
        with open(anno_file, 'r') as f:
            lines = f.readlines()

        # Extract the coordinates of all the annotated areas
        class_coords = {}
        for line in lines:
            line = line.strip()
            if line != '':
                class_id, x1, y1, x2, y2 = [int(x) for x in line.split()]
                if class_id not in class_coords:
                    class_coords[class_id] = []
                class_coords[class_id].append((x1, y1, x2, y2))

        # Calculate the mode of the pixel values for each class separately
        file_results = {}
        for class_id, coords in class_coords.items():
            class_mode_vals = []
            class_bbox_areas = []
            for x1, y1, x2, y2 in coords:
                # Load the Midas depth image
                midas_img = cv2.imread(midas_img_path)
                midas_img = cv2.cvtColor(midas_img, cv2.COLOR_BGR2RGB)

                # Preprocess the image for the Midas model
                input_batch = transform(midas_img).to(device)

                # Make a prediction using the Midas model
                with torch.no_grad():
                    prediction = midas(input_batch)

                    prediction = torch.nn.functional.interpolate(
                        prediction.unsqueeze(1),
                        size=midas_img.shape[:2],
                        mode="bicubic",
                        align_corners=False,
                    ).squeeze()

                # Convert the prediction to a numpy array
                midas_output = prediction.cpu().numpy()

                # Crop the corresponding region from the Midas output
                midas_roi = midas_output[y1:y2, x1:x2]

                # Calculate the mode of the pixel values in the ROI
                mode_val = np.argmax(np.bincount(midas_roi.flatten()))
                class_mode_vals.append(mode_val)

                # Calculate the area of the bounding box
                bbox_area = bbox_area = (x2 - x1) * (y2 - y1)
            class_bbox_areas.append(bbox_area)

        # Calculate the mode value and area of the bounding box for the class
        mode_val = np.argmax(np.bincount(class_mode_vals))
        bbox_area = np.median(class_bbox_areas)
        class_results = {'mode_value': mode_val, 'bbox_area': bbox_area}

        # Calculate the weight of the class
        class_weight = scale_factor * density[class_id] * bbox_area * mode_val
        class_results['weight'] = class_weight

        # Add the weight of the class to the total weight of the image
        total_weight += class_weight

        file_results[class_id] = class_results

    # Store the results in the dictionary
    results[filename] = file_results

print(results)